## 作業目標
    
建立一個網路模型

## 作業重點

請修改 Name 中, 自定義的 Layer 名稱, 並增加一層全連階層。

宣告 model API, 分別採用自行定義的 Input/Output Layer, 使用 model.summary 查看 Layers stack。

## 參考資料

### First example: a densely-connected network

The Sequential model is probably a better choice to implement such a network, but it helps to start with something really simple.

- A layer instance is callable (on a tensor), and it returns a tensor
- Input tensor(s) and output tensor(s) can then be used to define a Model
- Such a model can be trained just like Keras Sequential models.

[Getting started with the Keras functional API](https://keras.io/getting-started/functional-api-guide/)

In [1]:
import keras
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Dense
tf.logging.set_verbosity(tf.logging.ERROR)

Using TensorFlow backend.


In [2]:
news_in = Input(shape=(100,), dtype='int32', name='news_in')
x_out = Embedding(output_dim=512, input_dim=10000, input_length=100)(news_in)
lstm_out = LSTM(32)(x_out)
auxiliary_out = Dense(1, activation='sigmoid', name='auxiliary_out')(lstm_out)

In [3]:
auxiliary_in = Input(shape=(5,), name='auxiliary_in')
x_out = keras.layers.concatenate([lstm_out, auxiliary_in])
x_out = Dense(64, activation='relu')(x_out)
x_out = Dense(64, activation='relu')(x_out)
x_out = Dense(64, activation='relu')(x_out)
news_out = Dense(1, activation='sigmoid', name='news_out')(x_out)

In [4]:
model = Model(inputs=[news_in, auxiliary_in], outputs=[news_out, auxiliary_out])
model.compile(optimizer='rmsprop',
              loss={'news_out': 'binary_crossentropy', 'auxiliary_out': 'binary_crossentropy'},
              loss_weights={'news_out': 1., 'auxiliary_out': 0.2})

In [5]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
news_in (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 512)     5120000     news_in[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 32)           69760       embedding_1[0][0]                
__________________________________________________________________________________________________
auxiliary_in (InputLayer)       (None, 5)            0                                            
__________________________________________________________________________________________________
concatenat